# Components, mixtures and phase equilibrium model basics

SGTPy unit for temperature is [K], unit of pressure is [Pa] and unit of molar volume is [m^3/mol].

In [1]:
import numpy as np
from sgtpy import component, mixture, saftvrmie

## Single components

Molecular parameters are specified when a component is created with the ``component`` class function. Depending on the nature of the molecule different parameters will be needed:

- ``ms``: number of segments
- ``sigma``: shape parameter used in Mie potential [Amstrong]
- ``eps``: energy parameter used in Mie potential [K]
- ``lambda_a`` and ``lambda_r``: attractive and repulsive exponents in Mie Potential.
- ``ring``: geometric factor for ring-like molecules ([see Langmuir 2017, 33, 11518-11529, Table I.](https://pubs.acs.org/doi/abs/10.1021/acs.langmuir.7b00976))

If the molecule associates. the association scheme is characterized by the triple ``sites=[B, P, N]`` which indicates the number of bipolar, positive, and negative sites. Additionally, the following parameters are required:

- ``eAB``: self-association energy [K]
- ``rcAB`` and``rdAB``: association range and association site position in ``sigma`` units.

Finally, a polar contribution can be set by providing, the number of polar sites (``npol``) and the dipolar moment (``mupol``) in Debye.

Additionally, the influence parameter (``cii``) must be supplied for SGT calculations. The influence parameter will be evaluated as a temperature-dependent polynomial with the `numpy.polyval` function. For constant influence parameter (temperature-independent) just provide a float or a list/array of length 1.

Simple coarse-grained molecules can be defined as follows. Parameters for these molecules can be obtained from [Bottled Saft](https://pubs.acs.org/doi/10.1021/acs.jcim.6b00149).

In [2]:
# coarse grained methane and dodecane
methane = component('methane', ms=1.0, sigma=3.752 , eps=170.75, lambda_r=16.39, lambda_a=6.)

dodecane = component('dodecane', ms=4.0, sigma=4.351, eps=378.56, lambda_r=18.41, lambda_a=6.)

Molecular parameters can be obtained from the [corresponding states principle](https://pubs.acs.org/doi/abs/10.1021/ie404247e). The method ``component.safrvrmie_forcefield`` implements this correlation to compute the shape parameter (``sigma``), energy parameter (``eps``) and repulsive exponent (``lambda_r``) used in the Mie potential.

This method requires that the component has defined its critical temperature (``Tc``) and acentric factor (``w``). Additionally, it requires the number of segments and liquid density at Tr=0.7.

This method sets ``lambda_a=6``.

In [3]:
# creating molecule just with critical temperature (K) and acentric factor (w)
hexane = component('hexane', Tc=507.6, w=0.301261)

# the function requires the number of segments and the density at (Tr=0.7)
hexane.saftvrmie_forcefield(ms=2, rhol07=6973.5)
# sigma in meters, epsilon in K and lambda_r

(4.51074436184689e-10, 373.5197182392754, 19.436070493981134, 0.0)

Ring-like molecules are defined as follows:

In [4]:
benzene = component('benzene', ms=3, sigma=3.441, eps=230.30, lambda_r=10.45, lambda_a=6., ring=1.4938)

In the following code block self associating water (4C) and self associating- polar butanol are defined.

In [5]:
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515458e-20)


butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)

The molecular parameters of a component are saved as class attributes. They can be accesses or modified as regular python's object attributes.

In [6]:
print('Water ms: ', water.ms)
print('Water sigma: ', water.sigma, '(stored internally in meters)' )
print('Water eps: ', water.eps, '(stored internally in Joules)')
print('Water lambda_a: ', water.lambda_a)
print('Water lambda_r: ', water.lambda_r)
print('Water ring: ', water.ring )

print('Water sites: ', water.sites, '[B, P, N] triplet' )
print('Water rcAB: ', water.rcAB, '(stored internally in meters)' )
print('Water rdAB: ', water.rdAB, '(stored internally in meters)' )
print('Water eAB: ', water.eAB, '(stored internally in Joules)')

print('Water mupol: ', water.mupol )
print('Water npol: ', water.npol)

print('Water cii: ', water.cii)

Water ms:  1.7311
Water sigma:  2.4539e-10 (stored internally in meters)
Water eps:  1.5304491948e-21 (stored internally in Joules)
Water lambda_a:  6.0
Water lambda_r:  8.308
Water ring:  0.0
Water sites:  [0, 2, 2] [B, P, N] triplet
Water rcAB:  1.38007336e-10 (stored internally in meters)
Water rdAB:  9.815600000000002e-11 (stored internally in meters)
Water eAB:  2.748968406216e-20 (stored internally in Joules)
Water mupol:  0
Water npol:  0.0
Water cii:  [1.53719394e-20]


After a component is defined, it can be used with SAFT-VR-Mie eos through the ``saftvrmie`` function. 

In [7]:
# pure water with SAFT-VR-Mie
eos = saftvrmie(water)

## Mixtures

Mixtures are created from components with the ``mixture`` class function or by adding two pure components (`+`).

The mixture function allows setting interaction parameters, such as correction for the energy parameters used in cross interaction of Mie potential and the cross-interaction association energy. 

The cross-interactions parameters as defined as follows:

$$ \sigma_{ij} = \frac{\sigma_i + \sigma_j}{2} ;\quad
\epsilon_{ij} = (1-k_{ij}) \frac{\sqrt{\sigma_i^3 \sigma_j^3}}{\sigma_{ij}^3} \sqrt{\epsilon_i \epsilon_j} ;\quad
\lambda_{k,ij} = \sqrt{(\lambda_{k,i} - 3)(\lambda_{k,j} - 3)} + 3 \quad k=a,r $$

And for cross-associating mixtures: 

$$ r^{ABij}_c = \frac{r^{ABii}_c + r^{ABjj}_c}{2} ;\quad
r^{ABij}_d = \frac{r^{ABii}_d + r^{ABjj}_d}{2} ;\quad
\epsilon_{ij}^{AB} = (1 - l_{ij})\sqrt{\epsilon_{ii}^{AB} \epsilon_{jj}^{AB}} $$

Where the $k_{ij}$ and $l_{ij}$ interaction parameters are computed as:

$$
k_{ij} = k_{ij,0} + k_{ij,1} \cdot T +  k_{ij,2} \cdot T^2 + \frac{k_{ij,3}}{T} \\
l_{ij} = l_{ij,0} + l_{ij,1} \cdot T +  l_{ij,2} \cdot T^2 + \frac{l_{ij,3}}{T}
$$

There are two ways of setting the $k_{ij}$ and $l_{ij}$ interaction parameters. In the first one, you need to provide a symmetric matrix of shape (`nc, nc`) to the ``mixture.kij_saft`` and ``mixture.lij_saft`` methods. In the second method, you can provide an unique pair of $k_{ij}$ or $l_{ij}$ using the `mixture.set_kijsaft` or `mixture.set_lijsaft`.

In boths methods, by default only the $k_{ij, 0}$ and $l_{ij, 0}$ are required. The temperature dependent parameters are optional, if they are not provided they are assumed to be zero.

In [8]:
methane = component('methane', ms = 1.0, sigma = 3.752 , eps = 170.75,
                    lambda_r = 16.39, lambda_a = 6.)

dodecane = component('dodecane', ms = 4.0, sigma = 4.351 , eps = 378.56,
                    lambda_r = 18.41, lambda_a = 6.)

mix = mixture(methane, dodecane)
# or
mix = methane + dodecane

# interaction parameter optimized from phase equilibria data
kij = -0.02199102576365056
Kij = np.array([[0., kij], [kij, 0.]])
mix.kij_saft(Kij)

# or setting interaction between component i=0 (methane) and j=1 (dodecane)
mix.set_kijsaft(i=0, j=1, kij0=kij)

# you can access to the mixture attributes
print('Mixtures segments number array: ', mix.ms)
print('Mixture shape parameters array: ', mix.sigma)
print('Mixtures dipolar moment array: ', mix.mupol)


Mixtures segments number array:  [1.0, 4.0]
Mixture shape parameters array:  [3.752e-10, 4.3510000000000003e-10]
Mixtures dipolar moment array:  [0, 0]


For mixtures that exhibit cross-association, such as, the mixture of water and butanol you can set an ``kij`` correction for the cross-interaction energy of the Mie potential and an ``lij`` correction for the unlike association energy between the fluids.

In [9]:
# creating pure components
water = component('water', ms = 1.7311, sigma = 2.4539 , eps = 110.85,
                    lambda_r = 8.308, lambda_a = 6., eAB = 1991.07, rcAB = 0.5624,
                    rdAB = 0.4, sites = [0,2,2], cii = 1.5371939421515458e-20)

butanol = component('butanol2C', ms = 1.9651, sigma = 4.1077 , eps = 277.892,
                    lambda_r = 10.6689, lambda_a = 6., eAB = 3300.0, rcAB = 0.2615,
                    rdAB = 0.4, sites = [1,0,1], npol = 1.45, mupol = 1.6609,
                    cii  = 1.5018715324070352e-19)

mix = mixture(water, butanol)
# or
mix = water + butanol

# optimized from experimental LLE
kij, lij = np.array([-0.00736075, -0.00737153])

# setting interaction corrections
Kij = np.array([[0, kij], [kij, 0]])
Lij = np.array([[0., lij], [lij, 0]])
mix.kij_saft(Kij)
mix.lij_saft(Lij)

# or setting interaction between component i=0 (water) and j=1 (butanol)
mix.set_kijsaft(i=0, j=1, kij0=kij)
mix.set_lijsaft(i=0, j=1, lij0=lij)

SGTPy doesn't limit the size of the mixture. Further components can be added with the ``mixture.add_component`` method or by adding them to the mixture (`+`).

In [10]:
ethanol = component('ethanol2C', ms = 1.7728, sigma = 3.5592 , eps = 224.50,
                    lambda_r = 11.319, lambda_a = 6., eAB = 3018.05, rcAB = 0.3547,
                    rdAB = 0.4, sites = [1,0,1], cii= 5.3141080872882285e-20)

cpme = component('cpme', ms =  2.32521144, sigma = 4.13606074, eps = 343.91193798, lambda_r = 14.15484877, 
                 lambda_a = 6.0, npol = 1.91990385,mupol = 1.27, sites=[0,0,1], cii = 3.5213681817448466e-19)

hexane = component('hexane', ms = 1.96720036, sigma = 4.54762477, eps = 377.60127994, 
                   lambda_r = 18.41193194, npol = 0., cii = 3.581510586936205e-19 )

# creating mixture
mix = mixture(hexane, ethanol)
# adding a component
mix.add_component(cpme)

# creating a ternary mixture by adding the pure components
mix = hexane + ethanol + cpme

# adding a component to an existing mixture
mix = mixture(hexane, ethanol)
mix += cpme

Similarly as for pure fluids, once the mixture and its interaction parameters have been set up, the mixture can be used with the SAFT-VR-Mie EoS with the ``saftvrmie`` function.

In [11]:
eos = saftvrmie(mix)

/Users/gac21/Documents/sgtpy/sgtpy/vrmie_pure/ideal.py:25: RuntimeWarning: invalid value encountered in log
  a = np.log(rho * broglie_vol**3) - 1


The interactions of the mixture Hexane, Ethanol and CPME differ from the mixtures studied above because in this case CPME (1 negative site) is allowed to associate with Ethanol but not to self-associate.

This induced association phenomena can be modeled fitting the association volume between Ethanol and CPME and setting the unlike association energy as the half of the self-associating molecule (Ethanol in this case). This modification is can be done manually to the ``eos.rcij`` and ``eos.eABij`` attributes or by using the `eos.set_induced_asso` method.

The ``kij`` corrections are set up as usual with the ``mixture.kij_saft`` or `mixture.set_kijsaft` methods.

In [12]:
# creating mixture
mix = mixture(hexane, ethanol)
# adding a component
mix.add_component(cpme)

eos = saftvrmie(mix)

# setting kij correction matrix
k12 = 0.011818492037463553
k13 = 0.0008700151297528677
k23 =  0.01015194
Kij = np.array([[0., k12, k13], [k12, 0., k23], [k13, k23, 0.]])
mix.kij_saft(Kij)

# or by setting the kij interactions by pairs
mix.set_kijsaft(i=0, j=1, kij0=k12)
mix.set_kijsaft(i=0, j=2, kij0=k13)
mix.set_kijsaft(i=1, j=2, kij0=k23)

eos = saftvrmie(mix)

# manually cross associationg set up 
rc = 2.23153033 # Amstrong
eos.eABij[1,2] = ethanol.eAB / 2
eos.eABij[2,1] = ethanol.eAB / 2
eos.rcij[1,2] = rc * 1e-10
eos.rcij[2,1] = rc * 1e-10

# or by using the eos._set_induced_asso method selfasso=1 (ethanol), inducedasso=2 (cpme)
rc = 2.23153033 # Amstrong
eos.set_induced_asso(selfasso=1, inducedasso=2, rcij=rc)

---
For further information about each function/method check out the documentation running: ``function?``